# A computational companion to Alan Turing's paper *On the Chemical Basis of Morphogenesis* (1952)


In [ ]:
import numpy as np
from scipy.optimize import root_scalar

np.set_printoptions(formatter={'float_kind':"{:.2f}".format})
from sympy import *
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib notebook
from ipywidgets import interactive
import ipywidgets as widgets

# 3. Chemical Reaction
Suppose the reaction
$ A \rightarrow B $
is catalysed by G so we have
$ A + G \rightleftharpoons C \rightarrow B + G $

By the law of mass action,

$\begin{align}
    \frac{d[A]}{dt} &= -k_1[A][G] + k_2[C] \\
    \frac{d[B]}{dt} &= k_3[C] \\
    \frac{d[G]}{dt} &= -k_1[A][G] + k_2[C] + k_3[C] \\
    \frac{d[C]}{dt} &= k_1[A][G]-k_2[C] -k_3[C]
\end{align}$


reaction graph diagram here

Turing points out we can simplify these kinetics if $k_3$ is small, so $ C \rightarrow B + G $ is much slower than $A + G \rightleftharpoons C$.
In that case we have an approximate equilibrium where 
$\frac{d[A]}{dt} \approx \frac{d[G]}{dt} \approx 0 \approx -k_1[A][G] + k_2[C]$

so
$[A][G] \approx \frac{k_2}{k_1}[C]$

and $\frac{d[B]}{dt} = k_3[C] \approx \frac{k_1 k_3}{k_2}[A][G] $ 

The interactive plot below allows you to experiment with altering the parameters and initial concentrations of this system. Notice how when $k_2$ and $k_3$ are small almost all the catalyst is in the combined form so the reaction proceeds at rate determined by $[G]$ and independent of $[A]$.

In [ ]:
plt.close()
ax = plt.subplot()
def plot(k1,k2,k3,A0,G0):
    def sys(t, y):
        A, B, G, C = y
        diffA = -k1 * A * G + k2 * C
        diffB = k3 * C
        diffG = diffA + diffB
        diffC = -diffG
        return np.array([diffA, diffB, diffG, diffC])
    sol = solve_ivp(sys, (0, 50), (A0, 0, G0, 0), dense_output=True).sol
    t=np.linspace(sol.t_max,sol.t_min,50)
    y=sol(t)
    ax.clear()
    ax.set_ylim(0.0,1.0)
    ax.plot(t,y[0], 'r-')
    ax.plot(t,y[1], 'b-')
    ax.plot(t,y[2], 'y--')
    ax.plot(t,y[3], color='orange', linestyle='--')
    ax.legend(('A: reactant', 'B: product', 'G: catalyst', 'C: intermediate'))
    ax.xlabel('Time')
    ax.ylabel('Concentration')
    
k1_slider = widgets.FloatSlider(description='$k_1$', max=5, step=0.01, value=0.8)
k2_slider = widgets.FloatSlider(description='$k_2$', max=5, step=0.01, value=0)
k3_slider = widgets.FloatSlider(description='$k_3$', max=5, step=0.01, value=0.1)
A0_slider = widgets.FloatSlider(description='$[A]_0$', max=1, step=0.01, value=1.0)
G0_slider = widgets.FloatSlider(description='$[G]_0$', max=1, step=0.01, value=0.2)
#plt.close()
interactive_plot=interactive(plot, k1=k1_slider, k2=k2_slider, k3=k3_slider, A0=A0_slider, G0=G0_slider)
output = interactive_plot.children[-1]
output.layout.height = '450px'
interactive_plot


## 4. The breakdown of symmetry and homogeneity
Here a pair of cells containing similar concentrations of morphogens $X$ and $Y$ are in contact. The system is initially symmetrical - interchanging the cells results in the same dynamics.
The morphogen $X$ is produced at the rate

$5x -6y +1$

and $Y$ at the rate

$6x -7y +1$,

where $x$ and $y$ are the concentrations of $X$ and $Y$.

At the same time $X$ diffuses between the cells at the rate

$0.5(x_1-x_2)$

and $Y$ at the rate

$4.5(y_1-y_2)$.

Then we have

$\begin{align}
    \dot x_1 &= 5x_1 -6y_1 + 1 - 0.5(x_1-x_2) \\
    \dot x_2 &= 5x_2 -6y_2 + 1 + 0.5(x_1-x_2) \\
    \dot y_1 &= 6x_1 - 7y_1 + 1 - 4.5(y_1-y_2) \\
    \dot y_2 &= 6x_2 - 7y_2 + 1 + 4.5(y_1-y_2) \\
\end{align}$

Symmetry persists only if the concentrations in the two cells remain exactly identical, a situation impossible in nature. Here we solve the above system of equations with a tiny random difference introduced between the initial conditions for each cell.

In [ ]:
plt.close()
ax = plt.subplot()
Dx=0.5
Dy=4.5
I=0.2

def symmetry_broken(t,y):
    return np.abs(y[0]-y[1])-y[0]/2
    
# symmetry_broken.terminal = True

rng = np.random.default_rng()
y0 = rng.normal(1,0.0001,4)

print(f"x1 = {y0[0]}, x2 = {y0[1]}, y1 = {y0[2]}, y2 = {y0[3]}")

def plot(I):
    A = np.array([[3+I-Dx, Dx,  -6.0,  0.0],
                  [Dx,  3+I-Dx,   0.0, -6.0],
                  [6.0,  0.0, -9+I-Dy,  Dy],
                  [0.0,  6.0,   Dy, -9+I-Dy]])
    B = np.array([I-1,I-1,-I+1,-I+1])
    
    print(f"Eigenvalues: {np.real(np.linalg.eigvals(A)):.2f}")

    def sys(t, y):
        dy = A @ y + B 
        # Don't let concentration go below zero.
        return np.where(y>0, dy, np.maximum(0,dy))
   
    sol = solve_ivp(sys, (0, 6), y0, dense_output=True,
                    events=symmetry_broken).sol
    t=np.linspace(sol.t_max,sol.t_min,100)
    y=sol(t)
    ax.clear()
    ax.plot(t,y[0], 'r-')
    ax.plot(t,y[1], 'b-')
    ax.legend(('x1', 'x2'))
    ax.set_xlabel('Time')
    ax.set_ylabel('Concentration');
    
I_slider = widgets.FloatSlider(description='$I$', min=-1, max=3, step=0.01, value=2.0)
#plt.close()
interactive_plot=interactive(plot, I=I_slider)
output = interactive_plot.children[-1]
output.layout.height = '450px'
interactive_plot

## 6. Reactions and diffusion in a ring of cells
In this example $N$ cells are arranged in a ring. There are two morphogens $X$ and $Y$ as before, produced at rates $ax + by$ and $cx + dy$ respectively.

For neighbouring cells, $X$ and $Y$ have diffusion coefficients of $\mu$ and $\nu$.

This can be described by the set of equations

$\dot x_r = ax_r + by_r + \mu(x_{r+1} - 2x_{r-1} + x_r) $

$ \dot y_r = cx_r + dy_r + \nu(y_{r+1} - 2y_{r-1} + y_r) $

for $r = 1,...,N$.

The ring of cells if effectively an N-periodic sequence, so the equations can be solved exactly by taking the discrete fourier transform (DFT).

$\xi_r = \sum_{s=1}^N x_s e^{-i 2 \pi r s / N}$

$\eta_r = \sum_{s=1}^N y_s e^{-i 2 \pi r s / N}$

Differentiating and substituting in $\dot x$ and $\dot y$ gives

$\dot \xi_r = (a- 4 \mu sin^2 \frac{\pi r}{N}) \xi_r + b \eta_r$

$\dot \eta_r = (d- 4 \nu sin^2 \frac{\pi r}{N})\eta_r + c \xi_r$

These are pairs of linear differential equations which can be solved to give

$ \xi_s = A_s e^{p_st} + B_s e^{p_s' t} $

$ \eta_s = C_s e^{p_st} + D_s e^{p_s' t} $

where
$p_s$ and $p_s'$ are the roots of

$(p -a+4\mu sin^2 \frac{\pi s}{N})(p-d+4\nu sin^2 \frac{\pi s}{N}) = bc$

and $A_s, B_s, C_s, D_s$ are determined from the initial conditions so that

$ A_s(p_s - a +4 \mu sin^2 \frac{\pi s}{N}) = b C_s $

$ B_s(p_s' - a +4 \mu sin^2 \frac{\pi s}{N}) = b D_s $

We can now take the reverse DFT to find the general solution:

$ x_r = \sum_{s=1}^N(A_s e^{p_s t} + B_s e^{p_s' t}) e^{i 2\pi r s/N}$

$ y_r = \sum_{s=1}^N(C_s e^{p_s t} + D_s e^{p_s' t}) e^{i 2\pi r s/N}$

To find $A_s, B_s, C_s, D_s$ we set the inital conditions $x_0$ and $y_0$ and solve the linear equations

$ x_{r0} = h + \sum_{s=1}^N(A_s + B_s) e^{i 2\pi r s/N}$

$ y_{r0} = k + \sum_{s=1}^N(C_s + D_s) e^{i 2\pi r s/N}$

$ A_s(p_s - a +4 \mu sin^2 \frac{\pi s}{N}) = b C_s $

$ B_s(p_s - a +4 \mu sin^2 \frac{\pi s}{N}) = b D_s $

In [70]:
p,p_prime=fsolve(np.sin,[0,2])

In [78]:
a=np.arange(3)
b=np.arange(3)
a+b
2*a +b

[1, 2, 3, 1, 2, 3, 5, 6, 7]

In [82]:
%matplotlib inline


def plot_ring(x):
    n = len(x)
    theta = np.linspace(0, 2 * np.pi, n, endpoint=False)
    width = 2 * np.pi / n

    # Normalize values for coloring
    norm = plt.Normalize(min(x), max(x))
    colours = cm.viridis(norm(values))  # Colormap for the ring
    fig, ax = plt.subplots(subplot_kw=dict(polar=True))
    bars = ax.bar(theta, 1*n, width=width, color=colors, bottom=5*n)
    ax.set_yticklabels([])  # Remove radial ticks
    ax.set_xticks([])       # Remove angular ticks
    ax.grid(False)
    ax.spines['polar'].set_visible(False)  # Remove border
    plt.show()
    
values = np.random.rand(N) 
plot_ring(values)
N=20
x0 = np.random.rand(N)
y0 = np.random.rand(N) 


def plot(a,b,c,d,mu,nu):
    s = np.arange(N)
    p = a + 4 * mu *sin(pi * s/N)**2
    p_prime = d+4 * nu * sin(pi * s/N)**2
    # Build a matrix to solve the linear equations to find A,B,C,D
    z = np.fromfunction(lambda s,r: np.exp(1j*2*pi*r*s/N), (N,N), dtype=complex)
    q = p - a + 4 * mu * sin(pi * s/N)
    q_prime = p_prime - a + 4 * mu * sin(pi * s/N)
    mat = np.vstack((np.hstack((z,   z)),
                     np.hstack((q*z/b, q_prime*z/b))))
    ords = np.concatenate(x0,y0,np.zeros(2*N))
    sol = np.linalg.solve(mat,ords)
    A = sol[:N/2]
    B = sol[N/2:]
    C = q*A/b
    D = q_prime*B/b
    
    x = np.fromfunction(lambda r: np.sum(A*np.exp(p*t)+B*np.exp(p_prime*t)*np.exp(1j*2*pi*r*s/N)), N)
    y = np.fromfunction(lambda r: np.sum(C*np.exp(p*t)+D*np.exp(p_prime*t)*np.exp(1j*2*pi*r*s/N)), N)
    plot_ring(x)
for t in range(1000):
    plot(t,0.7,0.5,0,2,0.5,1)

TypeError: 'function' object cannot be interpreted as an integer